In [1]:
import torch
import torch.nn as nn
import math
import numpy as np
import datetime

np.random.seed(42)

import sys
print(sys.executable) # just to check which python

import gym
from gym import spaces

from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy

/usr/local/opt/python@3.9/bin/python3.9


In [2]:
class RequestType:
    def __init__(self, request_type, bandwidth, service_rate, arrival_rate, source, sink, distribution, switch_rate=None):
        # distribution is 1x2 if elastic and 1x1 if static
        
        self.type = request_type
        self.bw = bandwidth
        self.service_rate = service_rate
        self.arrival_rate = arrival_rate
        self.source = source
        self.sink = sink
        self.distribution = distribution
        self.switch_rate = switch_rate
        
        self.num_made = 0
        self.num_accepted = 0

class Request:
    def __init__(self, request_type, service_time, arrival_time, source, sink, transfer_rate, distribution=None, parent_elastic=None, bw_dist=None, request_type_template=None):
        self.type = request_type
        self.service_time = service_time
        self.arrival_time = arrival_time
        self.source = source
        self.sink = sink
        self.bw = transfer_rate
        self.request_type = request_type
        self.parent_elastic = parent_elastic
        self.accepted = None
        self.path = None
        self.bw_dist = bw_dist
        
        self.blueprint = request_type_template
        
        if request_type == "elastic":
            self.distribution = distribution
            self.scale_requests = []
                   
        if request_type_template is not None:
            request_type_template.num_made += 1
            
    def add_scale_request(self, req): 
        # we store related scale requests for elastic requests
        # not used if static request
        self.scale_requests.append(req)
            
    def get_encoding(self, nodes_in_environment):
        # as per our notes, this SHOULD return 1x5 tensor,
        # but we have one hot encodings INSIDE this tensor,
        # so we will flatten this and return, so the size will be
        # larger than 1x5
        
        # nodes_in_environment is a list of all the nodes in our graph
        # eg ["a", "b", "c"]
        
        # request is [one hot source, one hot destination, bw, service time, one hot type]
                
        one_hot_source = nn.functional.one_hot(torch.tensor([nodes_in_environment.index(self.source)]), num_classes=len(nodes_in_environment)).flatten()
        one_hot_dest   = nn.functional.one_hot(torch.tensor([nodes_in_environment.index(self.sink)]), num_classes=len(nodes_in_environment)).flatten()
    
        if self.request_type == "static":
            one_hot_type = torch.tensor([1, 0, 0])
        elif self.request_type == "elastic":
            one_hot_type = torch.tensor([0, 1, 0])
        elif self.request_type == "scale":
            one_hot_type = torch.tensor([0, 0, 1])
            
        encoding = torch.cat([one_hot_source, 
                             one_hot_dest,
                             torch.tensor([self.bw]), 
                             torch.tensor([self.service_time]),
                             one_hot_type])
        
        return encoding

In [3]:
class Link:
    def __init__(self, node_1, node_2, bw_capacity):
        self.serving_requests = []
        self.nodes = [node_1, node_2]
        self.total_bw = bw_capacity
        
    def reset(self):
        self.serving_requests = []
        
    def add_request(self, request_obj):
        self.serving_requests.append(request_obj)
        
    def remove_request(self, request_obj):
        self.serving_requests.remove(request_obj)
        
    def remaining_bw(self): 
        # subtracting bw being used from total bw capacity
        bw_being_used = 0
        for req in self.serving_requests:
            bw_being_used += req.bw
            
        return (self.total_bw - bw_being_used)

In [4]:
class Environment(gym.Env):
    # requests_in_service_encoder = nn.RNN(????, 7)
    metadata = {'render.modes': ['human']}
    
    def __init__(self, nodes, links, request_blueprints, use_RNN=False, sb3_compat=False):
        super(Environment, self).__init__()
                
        """
        nodes: list of strings where each string is just a name or identifier of a node
        links: list of tuples where in tuple t, t[0] is first node, t[1] is another node, and t[2] is bw capacity of the link
        request_blueprints: list of DeploymentRequest objects
        """
        self.nodes = nodes
        self.links = {}
        self.request_history = []
        self.E_history = []
        self.past_distributions = []
        self.request_blueprints = request_blueprints
        self.last_time = 0
        self.episode_timesteps = 600
        self.use_RNN = use_RNN
        self.sb3_compat = sb3_compat
        self.precomputed_paths = {}
        
        if sb3_compat:
            self.request_being_considered = None
        
        for link in links:
            if link[0] not in self.nodes or link[1] not in self.nodes:
                raise Exception("Node in link " + str(link) + " doesn't exist")
            
            link_obj = Link(*link)

            self.links[link[0] + link[1]] = link_obj
            self.links[link[1] + link[0]] = link_obj
            
        self.request_list = self.create_requests()
        self.request_queue = iter(self.request_list)
        
        # Setup gym-specific code
        env_encoding_size = self.get_encoding(increment_iterator=False).size()
        req_encoding_size = self.request_list[0].get_encoding(self.nodes).size()
        # print(env_encoding_size[0] + req_encoding_size[0])
        
        self.action_space = spaces.Box(low=0, high=math.inf,
                                      shape=(4,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-math.inf, high=math.inf,
                                      shape=(env_encoding_size[0] + req_encoding_size[0],), dtype=np.float32)
        
        # TODO, WRITE RNN logic
        
        #if use_RNN:
        #    self.requests_in_service_encoder = nn.RNN
        
    def precompute_paths(self):
        for req_type in self.request_blueprints:
            self.precomputed_paths[req_type.source + req_type.sink] = env.search(req_type.source, req_type.sink, [], [])
            
    def add_request(self, request, path=None): # we want to add this request to a link or path
        # path: a list of nodes that the request traverses including source and sink
        # if no path is specified, path is assumed to be [req.source, req.sink]
        
        if path is not None: 
            nodes = [[path[i], path[i + 1]] for i in range(len(path) - 1)]
            for node_pair in nodes:
                env.links[node_pair[0] + node_pair[1]].add_request(request)
        
        else:
            self.links[request.source + request.sink].add_request(request)
        
        request.accepted = True
        request.blueprint.num_accepted += 1
        self.request_history.append(request)
        # print(self.links[request.source + request.sink])
    
    def reset(self):
        for link in self.links.values():
            link.reset()
        self.request_history = []
        self.E_history = []
        self.past_distributions = []
        self.last_time = 0
        self.request_list = env.create_requests()
        self.request_queue = iter(self.request_list)
        
        return env.get_encoding()
        
    def reward(self, request, decision):
        base_rate = 1         # 1 when static
        type_bonus = 0.9      # 0.9 when static
        bw = request.bw
        if request.type == "elastic":
            #base_rate = request.bw
            type_bonus = 1.1                # 1.1 when elastic
            bw = np.array(request.bw_dist).dot(request.distribution)
            
        r = bw * base_rate * request.service_time * type_bonus
        
        # if remaining bandwidth on link(s) < 0, very "bad" reward
        if request.path is not None:
            path_length = len(request.path)
            
            r *= math.pow(0.9, path_length - 2)
            
            nodes = [[request.path[i], request.path[i + 1]] for i in range(len(request.path) - 1)]
            for node_pair in nodes:
                if self.links[node_pair[0] + node_pair[1]].remaining_bw() < 0:
                    print("exceeded: " + str(-r * 10))
                    return (-r * 10)
        else:
            # path is direct, so no decrease of reward needed
            remaining_bw = self.links[request.source + request.sink].remaining_bw()
            if remaining_bw < 0:
                print("exceeded: " + str(-r * 10))
                return (-r * 10)
        
        if decision == "accept":
            return r
        
        if decision == "reject":
            if request.type == "static" or request.type == "elastic":
                return 0
            elif request.type == "scale":
                if len(self.past_distributions) == 0:
                    return -1 * r
                
                else:
                    current_sum = torch.from_numpy(np.sum(self.past_distributions, axis=0))

                    average_past_distribution = current_sum / len(self.past_distributions)
                    current_req_distribution = torch.tensor(request.parent_elastic.distribution)

                    return -1 * r * math.exp(-nn.functional.kl_div(average_past_distribution, current_req_distribution))

                """
                past_distributions = []
                for req in self.request_history:
                    if req.request_type == "elastic":
                        past_distributions.append(req.distribution)
                
                average_past_distribution = torch.mean(past_distributions, dim=1)
                current_req_distribution = torch.tensor(request.distribution)
                
                if bool(average_past_distribution[0] < current_req_distribution[0]):
                    return -1 * r * math.exp(-nn.functional.kl_div(average_past_distribution, current_req_distribution))
                else:
                    return 0
                """
                
    def next_req(self):
        next_req = next(self.request_queue)
        if self.sb3_compat:
            self.request_being_considered = next_req
        return next_req
                
    def step(self, action, req=None):
        # what happens if we have two requests that come in on the same timestep but there is only enough bandwidth for one?
        # do we the decision on the second request with knowledge of the first request
        # essentially, after we accept the first request, will we submit an updated encoding of the network to the policy network?
 
        # actions is a Nx2 matrix where the first column in the request and second is the decision
        # decision is either "accept" or "reject"
        # this is given by our agent
        
        # if req is None, that means we are using sb3_compat=True and we can get the req from self.request_being_considered
                
        if req is None:
            req = self.request_being_considered
            
        if action[0] > 0.5:
            # accept request
            paths = env.precomputed_paths[req.source + req.sink]
            paths.sort(key=lambda x: len(x)) # sort by shortest path
            # select the path we are using
            path = paths[action[1:4].argmax()]
            
            self.add_request(req, path)
        
            reward = env.reward(req, "accept")
        elif action[0] < 0.5:
            # reject
            reward = env.reward(req, "reject")
        
        obs = env.get_encoding()
        
        done = req.arrival_time > 600
        info = {}
        
        return obs, reward, done, info
        
    def update_requests(self, current_time):
        # here, we remove expired requests and update E_history based off of the request stats
        
        for link in self.links.values():
            for request in link.serving_requests.copy():
                if (request.arrival_time + request.service_time) > self.last_time and (request.arrival_time + request.service_time) < current_time:
                    # request has expired, let's remove it from the links
                    for link in self.links.values():
                        if request in link.serving_requests:
                            link.remove_request(request)

                    if request.type == "elastic":
                        time_on_higher_bw = 0
                        for scale_req in request.scale_requests:
                            time_on_higher_bw += scale_req.service_time

                        time_on_lower_bw = request.service_time - time_on_higher_bw

                        # calculate E[history]
                        request_time = np.array([time_on_lower_bw, time_on_higher_bw])
                        request_bw = request.bw
                        result = (request_time / request_time.sum()).dot(request_bw)
                        self.past_distributions.append(request_time / request_time.sum())
                        self.E_history.append(result)

    def get_encoding(self, increment_iterator=True):
        links_processed = [] 
        # these will store links that we have already encoded so we don't encode them again
        
        current_encoding = []
        
        # h = torch.zeros(7) # assuming 7 for h0 size
        # last_out = None
        
        env_encoding = []
        
        if increment_iterator:
            next_req = self.next_req()

            while next_req.type == "scale":
                if not next_req.parent_elastic.accepted:
                    next_req = self.next_req()
                elif next_req.parent_elastic.accepted:
                    break
            
            """
            while next_req.type == "scale":
                if next_req.parent_elastic.accepted:
                    next_req.accepted = True # we must accept since we accepted elastic req
                    self.add_request(next_req, next_req.parent_elastic.path)
                next_req = self.next_req()
            """
            self.update_requests(next_req.arrival_time)

        for link in self.links.values():
            if link in links_processed:
                continue

                        
            # Commented because we don't want to encode any queue for phase 1
            
            # for req in link.serving_requests
                # request is [one hot source, one hot destination, bw, service time, one hot type]
                
                # one_hot_source = nn.functional.one_hot(torch.tensor([self.nodes.index(req.source)]), num_classes=len(self.nodes))
                # one_hot_dest   = nn.functional.one_hot(torch.tensor([self.nodes.index(req.sink)]), num_classes=len(self.nodes))

                # req_tensor = torch.Tensor([]) # mismatched dimensions??!
                # last_out, h = self.requests_in_service_encoder(req_tensor, h)

            # current_encoding.append(torch.cat(torch.Tensor([link.remaining_bw]), last_out))
            # torch.stack(current_encoding)
            
            # check implementation later
            
            env_encoding.append(link.remaining_bw())
            
            links_processed.append(link)
            
        if not increment_iterator:
            return torch.tensor(env_encoding)
        
        if self.sb3_compat:
            return torch.cat([torch.tensor(env_encoding), torch.tensor(next_req.get_encoding(self.nodes))])
        else:
            return torch.tensor(env_encoding), torch.tensor(next_req.get_encoding(self.nodes)), next_req
    
    def create_requests(self):
        requests = []
        
        for request_type in self.request_blueprints:
            arrival_times = []
            service_times = []
            last_arrival = 0
        
            while last_arrival < self.episode_timesteps: # we want to generate requests till we reach episode end
                last_arrival += np.random.exponential(request_type.arrival_rate)
                arrival_times.append(last_arrival)
                                
            for _ in arrival_times:
                service_times.append(np.random.exponential(request_type.service_rate))
                
            for arrival_time, service_time in zip(arrival_times, service_times):
                # start creating requests
                
                new_request = Request(request_type.type, service_time, arrival_time, request_type.source, request_type.sink, 
                                      request_type.bw[0], request_type.distribution, bw_dist=request_type.bw, request_type_template=request_type)
                requests.append(new_request)
                
                if request_type.type == "elastic": 
                    # we will start with the first bandwidth element as starting bw
                    # WE ASSUME that bw[0] < bw[1]
                    timesteps_from_deployment = 0
                    current_bw = request_type.bw[0]
                    while timesteps_from_deployment < service_time:
                        if current_bw == request_type.bw[0]:
                            # we want to generate a scale request to increase bw
                            scale_bw = request_type.bw[1] - current_bw
                            scale_service_time = np.random.exponential(request_type.switch_rate[1])
                            scale_request = Request("scale", scale_service_time, \
                                                    arrival_time + timesteps_from_deployment, request_type.source, \
                                                   request_type.sink, scale_bw, parent_elastic=new_request,
                                                   request_type_template=request_type)
                            requests.append(scale_request)
                            new_request.add_scale_request(scale_request)
                            
                            timesteps_from_deployment += scale_service_time
                            current_bw = request_type.bw[1] # request_type.bw[0] + scale_bw
                        elif current_bw == request_type.bw[1]:
                            # we want to go to lower bw and spend some time there
                            time_spent_on_lower_bw = np.random.exponential(request_type.switch_rate[0])
                            timesteps_from_deployment += time_spent_on_lower_bw
                            current_bw = request_type.bw[0]
                            
        # sort requests by arrival time
        requests.sort(key=lambda x: x.arrival_time)
        return requests
    
    def search(self, source, dest, visited_a, paths):
        visited_a.append(source)
        # print(visited_a)

        for link in set(env.links.values()):
            visited = visited_a.copy()
            if source in link.nodes:
                if dest in link.nodes:
                    visited.append(dest)
                    paths.append(visited)

                x = link.nodes.copy()
                x.remove(source)
                if x[0] not in visited:
                    self.search(x[0], dest, visited.copy(), paths)
        return paths
    
    def print_statistics(self):
        for req_type in self.request_blueprints:
            print(req_type.source + " | " +
                   req_type.sink + " | " + 
                 "BW: " + str(req_type.bw) + " | " +
                 "Arrival rate: " + str(req_type.arrival_rate) + " | " +
                 "Acceptance rate: " + str(req_type.num_accepted / req_type.num_made))
            
        resources_used = 0
        for req in self.request_list:
            if req.accepted == True:
                resources_used += req.bw * req.service_time
            
        print("BW used: " + str(resources_used))

In [5]:
env = Environment(["a", "b", "c", "d", "e", "f"], [["a", "b", 10], ["a", "c", 10], ["b", "d", 10], \
                                                   ["c", "d", 20], ["c", "e", 10], ["d", "f", 10], \
                                                   ["e", "f", 10]], \
                  [RequestType("static", [2], 0.5, 0.75, "a", "b", [1]), \
                  RequestType("static", [8], 1, 1.5, "a", "b", [1]), \
                  RequestType("elastic", [4, 9], 1, 1.5, "a", "b", [0.8, 0.2], switch_rate=[0.08, 0.02]), \
                  RequestType("static", [1], 1, 1.5, "c", "d", [1]), \
                  RequestType("static", [7], 0.5, 0.75, "c", "d", [1]), \
                  RequestType("elastic", [3, 13], 2, 3, "c", "d", [0.9, 0.1], switch_rate=[0.09, 0.01]), \
                  RequestType("static", [3], 0.5, 0.75, "e", "f", [1]), \
                   RequestType("static", [6], 1, 1.5, "e", "f", [1]), \
                    RequestType("elastic", [5, 8], 2, 3, "e", "f", [0.7, 0.3], switch_rate=[0.07, 0.03])],
                 sb3_compat=True)


                # self, request_type, bandwidth, service_rate, arrival_rate, source, sink, distribution, switch_rate=None
    
env.precompute_paths()

### Choose shortest viable path

In [6]:
def policy(env_encoding, next_req_encoding, next_req_obj):    
    # find all paths between source and sink
    paths = env.precomputed_paths[next_req_obj.source + next_req_obj.sink] 
    # (env.search(next_req_obj.source, next_req_obj.sink, [], []))
    paths.sort(key=lambda x: len(x)) # sort by shortest path
    selection = 0
    for path in paths:
        # check if this path works
        works = True
        nodes = [[path[i], path[i + 1]] for i in range(len(path) - 1)]
        for node_pair in nodes:
            if env.links[node_pair[0] + node_pair[1]].remaining_bw() < next_req_obj.bw:
                works = False
                
        if works:
            selection = paths.index(path)
            selection_one_hot = nn.functional.one_hot(torch.tensor([selection]), num_classes=3).flatten()
            next_req_obj.path = path
            return torch.cat([torch.tensor([1]), selection_one_hot])
        
    return torch.cat([torch.tensor([0]), torch.tensor([0,0,0])])

In [7]:
# ONLY WORKS WITH sb3_compat=False when creating env object

total_reward = 0
env_encoding, next_req_encoding, next_req_obj = env.reset()
done = False

while not done:
    decision = policy(env_encoding, next_req_encoding, next_req_obj)
    
    obs, reward, done, info = env.step(decision, next_req_obj)
    env_encoding, next_req_encoding, next_req_obj = obs
    
    total_reward += reward

print(total_reward)

<ipython-input-4-4b6285cd0530>:269: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(env_encoding), torch.tensor(next_req.get_encoding(self.nodes)), next_req
/usr/local/lib/python3.9/site-packages/torch/nn/functional.py:2610: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


11829.620237371953


In [8]:
env.print_statistics()

a | b | BW: [2] | Arrival rate: 0.75 | Acceptance rate: 0.48520345252774355
a | b | BW: [8] | Arrival rate: 1.5 | Acceptance rate: 0.2670967741935484
a | b | BW: [4, 9] | Arrival rate: 1.5 | Acceptance rate: 0.3766068099303307
c | d | BW: [1] | Arrival rate: 1.5 | Acceptance rate: 0.5038659793814433
c | d | BW: [7] | Arrival rate: 0.75 | Acceptance rate: 0.3832731648616125
c | d | BW: [3, 13] | Arrival rate: 3 | Acceptance rate: 0.23579912971892272
e | f | BW: [3] | Arrival rate: 0.75 | Acceptance rate: 0.4553686934023286
e | f | BW: [6] | Arrival rate: 1.5 | Acceptance rate: 0.3284132841328413
e | f | BW: [5, 8] | Arrival rate: 3 | Acceptance rate: 0.40795698924731183
BW used: 12529.47497216749


### PPO

In [6]:
model = PPO(MlpPolicy, env, verbose=1)
model.learn(total_timesteps=600000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


<ipython-input-4-3a01dc88a67e>:274: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.cat([torch.tensor(env_encoding), torch.tensor(next_req.get_encoding(self.nodes))])
/usr/local/lib/python3.9/site-packages/torch/nn/functional.py:2610: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


exceeded: -76.11723038718691
exceeded: -3.597510090968576
exceeded: -21.20412444609125
exceeded: -0.6575604339719677
exceeded: -93.64222533713806
exceeded: -1.525571636038893
exceeded: -0.5782132472615229
exceeded: -0.39402911056504986
exceeded: -0.24352947043225823
exceeded: -0.6367470720869921
exceeded: -1.0138355215000556
exceeded: -1.0710911086908839
exceeded: -49.72308306244169
exceeded: -88.24099588489429
exceeded: -172.46477246045006
exceeded: -12.342519766637963
exceeded: -33.659249112903126
exceeded: -100.52652605968534
exceeded: -331.4671474855761
exceeded: -0.09548062120440502
exceeded: -0.2117843945972941
exceeded: -99.07152992878274
exceeded: -45.15424794459871
exceeded: -0.40412531876885066
exceeded: -0.33978246317760136
exceeded: -0.9308292910462106
exceeded: -1.316701314548507
exceeded: -6.503329963030068
exceeded: -0.2604857057196127
exceeded: -1.6923065950576044
exceeded: -2.3638361650096043
exceeded: -8.174299148065344
exceeded: -10.236669036972874
exceeded: -0.30028

exceeded: -0.11667945645068045
-----------------------------------------
| time/                   |             |
|    fps                  | 1286        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008135144 |
|    clip_fraction        | 0.0835      |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.7        |
|    explained_variance   | 0.00608     |
|    learning_rate        | 0.0003      |
|    loss                 | 1.78e+03    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.012      |
|    std                  | 1.01        |
|    value_loss           | 3e+03       |
-----------------------------------------
exceeded: -0.31022261255174105
exceeded: -3.400906136200254
exceeded: -0.19233032596100064
exceeded: -0.9310975157638569
exceeded: -15.139289863990854
exceeded: -464.85725

exceeded: -3.203999235641
exceeded: -37.14668470648571
exceeded: -0.6647470340966113
exceeded: -6.7719727120918405
exceeded: -1.5337448935569842
exceeded: -26.296764886401583
exceeded: -28.247836642265398
exceeded: -11.469644542020864
exceeded: -4.8173732052385745
exceeded: -0.0011345666809514634
exceeded: -23.125785483426906
exceeded: -3.3362786197308125
exceeded: -2.084696237260892
exceeded: -2.703722620165018
exceeded: -70.87175224194313
exceeded: -0.45648442447898885
exceeded: -1.0300647402695204
exceeded: -2.295998250476709
exceeded: -233.24290406236167
exceeded: -28.41475509954435
exceeded: -61.160689357229465
exceeded: -38.77855411837084
exceeded: -78.46460797044523
exceeded: -1.3850957340528667
exceeded: -11.25652087207033
exceeded: -1.0236637640071116
exceeded: -41.255930662488424
exceeded: -1.9847615208653422
exceeded: -91.90276865073317
exceeded: -0.8995838502627195
exceeded: -1.3385175500685242
exceeded: -2.246923607487408
exceeded: -0.8410433114241747
exceeded: -0.77626802

...

exceeded: -0.14237778756066166
exceeded: -25.87163201422445
exceeded: -0.08914060123281822
exceeded: -0.38047414912321326
exceeded: -0.2519940449085514
exceeded: -7.3191986584082525
exceeded: -0.05437281138682429
exceeded: -2.5041453740598523
exceeded: -10.182663975118533
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.2e+04     |
|    ep_rew_mean          | 2.29e+03    |
| time/                   |             |
|    fps                  | 1035        |
|    iterations           | 275         |
|    time_elapsed         | 543         |
|    total_timesteps      | 563200      |
| train/                  |             |
|    approx_kl            | 0.012274327 |
|    clip_fraction        | 0.145       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.19       |
|    explained_variance   | 0.141       |
|    learning_rate        | 0.0003      |
|    loss                 | 2.44e+03    |
|    n_update

exceeded: -2.20703560483602
exceeded: -2.0330179828204193
exceeded: -205.26088826212833
exceeded: -1.1628855499119313
exceeded: -0.9148523123979845
exceeded: -0.14193386216232246
exceeded: -1.3298330368735976
exceeded: -2.5208165324207714
exceeded: -2.6335520444268745
exceeded: -0.15823889466597862
exceeded: -17.344760717725936
exceeded: -0.8790878043354331
exceeded: -0.299362785525208
exceeded: -3.2156708991201475
exceeded: -0.13577775016560234
exceeded: -5.481547536277214
exceeded: -0.04934070660673627
exceeded: -1.1527578574852775
exceeded: -1.1422683044527582
exceeded: -0.4223033224063961
exceeded: -0.5433330332768618
exceeded: -1.428187460755069
exceeded: -0.17990854426076922
exceeded: -0.18280616865468416
exceeded: -0.13355778415146208
exceeded: -0.8080993255695581
exceeded: -1.7293740750672608
exceeded: -345.571127486973
exceeded: -0.35647297111403586
exceeded: -0.7213921696288288
exceeded: -0.04532892761649338
exceeded: -0.019729537627441573
exceeded: -0.6863873667319315
exceed

exceeded: -1.9901495940919625
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.2e+04     |
|    ep_rew_mean          | 2.31e+03    |
| time/                   |             |
|    fps                  | 1037        |
|    iterations           | 284         |
|    time_elapsed         | 560         |
|    total_timesteps      | 581632      |
| train/                  |             |
|    approx_kl            | 0.021214604 |
|    clip_fraction        | 0.235       |
|    clip_range           | 0.2         |
|    entropy_loss         | -3.09       |
|    explained_variance   | 0.0708      |
|    learning_rate        | 0.0003      |
|    loss                 | 296         |
|    n_updates            | 2830        |
|    policy_gradient_loss | -0.0164     |
|    std                  | 0.533       |
|    value_loss           | 760         |
-----------------------------------------
exceeded: -116.8389219518927
exceeded: -2.7591

exceeded: -46.03327841620334
exceeded: -0.4379978912659837
exceeded: -0.32022130930670323
exceeded: -7.355724524864087
exceeded: -3.9738221472466444
exceeded: -0.8724035954914645
exceeded: -0.1086888108070973
exceeded: -0.5268881362970202
exceeded: -4.556758587571538
exceeded: -1.0216492493064209
exceeded: -0.5567836383124514
exceeded: -1.4422494594317423
exceeded: -0.175676126920064
exceeded: -0.6641090480504431
exceeded: -0.8507058114060652
exceeded: -1.3242013365766883
exceeded: -0.3297438677531834
exceeded: -0.26422483176961215
exceeded: -8.659368285685042
exceeded: -3.353838709734368
exceeded: -0.5986603987717736
exceeded: -4.027020543911865
exceeded: -1.3298198907425538
exceeded: -1.085740929589465
exceeded: -78.23581547873074
exceeded: -0.45648533962825366
exceeded: -3.1526541784233246
exceeded: -0.6130144116420481
exceeded: -1.4783587139815093
exceeded: -3.557580582603803
exceeded: -23.577386319664345
exceeded: -0.503156732860686
exceeded: -0.46694309481152085
exceeded: -0.6686

exceeded: -0.32071378722082483
exceeded: -0.06578106866367489
exceeded: -0.7567110696184058
exceeded: -182.85745511136895
exceeded: -0.194233546064146
exceeded: -1.0370707222659472
exceeded: -1.0014781421186945
exceeded: -2.8956537031459333
exceeded: -16.91952191578345
exceeded: -0.32876274119556964
exceeded: -0.3311203135327929
exceeded: -2.6448494503034197
exceeded: -0.4755654953516741
exceeded: -35.8357500180778
exceeded: -0.5233992679552439
exceeded: -0.39258894801952127
exceeded: -0.9923558492138753
exceeded: -0.4105998945685915
exceeded: -0.5650524765108432
exceeded: -1.0350397419904598
exceeded: -1.856946294199418
exceeded: -1.971050568347592
exceeded: -0.18824163264798008
exceeded: -0.09024887379882914
exceeded: -0.48480088964846635
exceeded: -0.3505423427222384
exceeded: -0.046784885581342354
exceeded: -2.902979152604735
exceeded: -24.732943987675682
exceeded: -0.4832301361094115
exceeded: -0.4492719816421725
exceeded: -0.08170689456408914
exceeded: -0.08477182533861886
exceed

In [8]:
total_reward = 0
observation = env.reset()
done = False
while not done:
    action, _states = model.predict(observation)
        
    observation, reward, done, info = env.step(action)
        
    total_reward += reward
    
print(total_reward)

<ipython-input-4-3a01dc88a67e>:274: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.cat([torch.tensor(env_encoding), torch.tensor(next_req.get_encoding(self.nodes))])


exceeded: -4.665130686013766
exceeded: -1.9536245346367898
exceeded: -7.877867497882596
exceeded: -192.71008086654976
exceeded: -1.5878856665312315
exceeded: -0.1412544469174888
exceeded: -0.2424055613866777
exceeded: -0.6162871322937709
exceeded: -2.208778564138499
exceeded: -0.18086723527172235
exceeded: -0.8025489212264398
exceeded: -0.4603388968658789
exceeded: -0.15715175486795135
exceeded: -2.219888236372896
exceeded: -0.5478824084612736
exceeded: -6.347802176238988
exceeded: -0.8073332115947769
exceeded: -0.5928143275961243
exceeded: -0.950266292882262
exceeded: -0.21829156851494824
exceeded: -26.79816321464013
exceeded: -5.578838775822996
exceeded: -0.4248945799279231
exceeded: -0.4880356308259536
exceeded: -0.2470708113930435
exceeded: -0.35435324953108827
exceeded: -0.6967848070966103
exceeded: -0.08367250716870611
exceeded: -0.8187467048732394
exceeded: -0.06905973139019997
exceeded: -1.2733695278473525
exceeded: -0.7335783723175836
exceeded: -0.504025034678973
exceeded: -0.

In [ ]:
env.print_statistics()

In [ ]:
model.learn(total_timesteps=100000)

In [ ]:
env.precomputed_paths

In [ ]:
today()